In [ ]:
# !pip install openai
!pip install -U torch transformers tokenizers accelerate safetensors

In [3]:
print(torch.__version__)
print(torch.cuda.is_available())

1.13.1+cpu
False


In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM

MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

pipe = pipeline(
    'text-generation', 
    model=model,
    tokenizer=MODEL,
    device=0
)

def ask(x, context='', is_input_full=False):
    ans = pipe(
        f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:", 
        do_sample=True, 
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    print(ans[0]['generated_text'])

In [8]:
from elasticsearch import helpers, Elasticsearch
import csv
import os
import requests
import pandas as pd
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from elasticsearch.helpers import reindex
# import openai
# requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
os.chdir('c:\\Users\\jieun\\Desktop\\project\\elastic')
os.getcwd()

'c:\\Users\\jieun\\Desktop\\project\\elastic'

In [6]:
es = Elasticsearch('https://172.16.204.219:9200', http_auth=['jieun','jieun123'], verify_certs=False)
es

<Elasticsearch([{'host': '172.16.204.219', 'port': 9200, 'use_ssl': True}])>

In [15]:
# with open('data/doc2_실시간도착_역정보(20230828).csv', 'rt', encoding='utf-8') as f:
#   print(f.read())
# #   fieldnames = ['search_string', 'sort']
#   reader = csv.DictReader(f)
#   helpers.bulk(es, reader, index='my-index2')

import pandas as pd

# CSV 파일 경로 설정
csv_file_path = '../data/tmp_fulltext.csv'  # 업로드할 CSV 파일의 경로를 지정하세요.
index_name = "fulltext-db"

# CSV 파일을 Pandas DataFrame으로 읽기
df = pd.read_csv(csv_file_path)

# DataFrame을 JSON 형식으로 변환 (각 행을 JSON 객체로 변환)
documents = df.to_dict(orient='records')

# Bulk API 요청 생성
bulk_data = []
for document in documents:
    # 각 문서를 인덱스와 타입과 함께 설정
    action = {
        "_index": index_name,  # Elasticsearch 인덱스 이름을 지정하세요.
        "_source": document
    }
    bulk_data.append(action)

# Bulk API를 사용하여 데이터 업로드
success, failed = helpers.bulk(es, bulk_data, index = index_name, raise_on_error=False)

print(f"업로드 성공: {success}, 업로드 실패: {failed}")

c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\jieun\anacond

업로드 성공: 2065, 업로드 실패: []


c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [16]:
index_list = es.indices.get_alias().keys()
for index in index_list:
    print(index)

.security-7
byte-image-index
rules
rules_folder
.kibana_task_manager_1
.apm-agent-configuration
.kibana_1
fulltext-db


c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [17]:
# 인덱스의 매핑 정보 가져오기
mapping_info = es.indices.get_mapping(index = index_name)

# 매핑 정보 출력
print(mapping_info)

{'fulltext-db': {'mappings': {'properties': {'document': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}}}


c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [ ]:
# 제거할 토크나이저 이름 지정
tokenizer_name_to_remove = "nori_sample"  # 제거하려는 토크나이저 이름을 설정하세요
index_name = "my-index2"  # 인덱스 이름을 적절히 변경하세요

# 인덱스를 닫습니다.
es.indices.close(index=index_name)

# 토크나이저를 포함한 설정에서 토크나이저를 제거합니다.
tokenizer_settings = {
    "settings": {
        "analysis": {
            "tokenizer": {
                tokenizer_name_to_remove: None  # 토크나이저를 제거하려면 None으로 설정합니다.
            }
        }
    }
}

# 설정 업데이트
es.indices.put_settings(index=index_name, body=tokenizer_settings)

# 인덱스를 다시 엽니다.
es.indices.open(index=index_name)

c:\Users\jieun\anaconda3\envs\pykg2vec\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'settings' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

In [23]:
# 삭제할 인덱스 이름 지정
index_name = "연구분야추가"  # 삭제하려는 인덱스 이름을 설정하세요

# 인덱스 삭제 명령 실행
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"인덱스 '{index_name}' 삭제 완료")
else:
    print(f"인덱스 '{index_name}'는 이미 존재하지 않습니다.")

인덱스 '연구분야추가' 삭제 완료


c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


---

In [21]:
# 검색할 쿼리 설정 (예: 모든 문서 검색)
query = {
    "query": {
        "match_all": {}
    }
}

# 인덱스에서 데이터 검색
search_results = es.search(index = index_name, body=query)

# 검색 결과 출력
for hit in search_results['hits']['hits']:
    print(hit['_source'])

{'document': "공과대학 기계정보공학과 김태현교수가 주관하는 연구는 2020에 농촌진흥청에서 주최하는 'IoT 기술을 이용한 최첨단 제어시설 확립을 위한 계획 수립안 제시'에 대한 연구이다. 이 연구는 물리/기계 측정표준에서 환경에 적용될 수 있다."}
{'document': "공과대학 기계정보공학과 김태현교수가 주관하는 연구는 2022-2023에 과학기술정보통신부에서 주최하는 '개방형 IoT·엣지 컴퓨팅 환경 기반 딥러닝 융합 오픈 소스 지능형 고장 진단 시스템 개발'에 대한 연구이다. 이 연구는 임베디드 S/W에서 제조업(전자부품, 컴퓨터, 영상, 음향 및 통신장비)에 적용될 수 있다."}
{'document': "공과대학 기계정보공학과 나영승교수가 주관하는 연구는 2020-2022에 과학기술정보통신부에서 주최하는 '고분자 전해질막 수전해 분리판 유로 및 확산체 설계'에 대한 연구이다. 이 연구는 유체기계에서 기타 산업에 적용될 수 있다."}
{'document': "공과대학 기계정보공학과 이광훈교수가 주관하는 연구는 2021에 과학기술정보통신부에서 주최하는 'X-TWICE 실전문제연구단'에 대한 연구이다. 이 연구는 S/W 솔루션에서 교육 및 인력양성에 적용될 수 있다."}
{'document': "공과대학 기계정보공학과 이광훈교수가 주관하는 연구는 2020에 교육부에서 주최하는 '대학혁신지원(R&D)'에 대한 연구이다. 이 연구는 과학기술과 인문사회에서 지식의 진보(비목적 연구)에 적용될 수 있다."}
{'document': "공과대학 기계정보공학과 이광훈교수가 주관하는 연구는 2020에 농촌진흥청에서 주최하는 '재생에너지 이용 및 자원의 순환을 통한 에너지 자립형 둔방형 첨단 R&BD 센터 운영안'에 대한 연구이다. 이 연구는 물리/기계 측정표준에서 환경에 적용될 수 있다."}
{'document': "공과대학 기계정보공학과 이광훈교수가 주관하는 연구는 2022-2023에 과학기술정보통신부에서 주최하는 '폐합성목재 촉매열처리공정을 통한 수소 

c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [8]:
# # 노리 토크나이저 및 필터 설정
# def configure_nori_tokenizer():
#     # 노리 토크나이저 및 필터 설정
#     tokenizer_settings = {
#         "settings": {
#             "analysis": {
#                 "tokenizer": {
#                     "my_nori_tokenizer": {
#                         "type": "nori_tokenizer",
#                         "decompound_mode": "mixed"
#                         # "user_dictionary": "userdict_ko.txt"
#                     }
#                 },
#                 "analyzer": {
#                     "my_nori_analyzer": {
#                         "type": "custom",
#                         "tokenizer": "my_nori_tokenizer",
#                         "filter": ["my_shingle"]
#                     }
#                 },
#                 "filter": {
#                     "my_shingle": {
#                         "type": "shingle",
#                         "token_separator": "",
#                         "max_shingle_size": 2
#                     }
#                 }
#             }
#         }
#     }
    
#     # 매핑에 노리 분석기 적용
#     es.indices.close(index="fulltext_db")  # 인덱스를 닫음 (인덱스를 수정하기 위해 필요)
    
#     # 설정 업데이트
#     es.indices.put_settings(index="fulltext_db", body=tokenizer_settings)
    
#     es.indices.open(index="fulltext_db")  # 인덱스를 다시 열음

# if __name__ == "__main__":
#     configure_nori_tokenizer()  # 노리 토크나이저 및 필터 설정

c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [25]:
###########################
## ES에 데이터 올리는 코드 ##
###########################

# CSV 파일 불러오기
# data_path = "data/"
# csv_files = os.listdir(data_path)
# dataframes = [pd.read_csv(data_path + file) for file in csv_files]
# print(csv_files)

df = pd.read_csv('C:/Users/jieun/Desktop/project/elastic/data/연구분야추가2.csv')
df = df.fillna(0)
index_name = "연구분야추가2"

field_name = df.columns.to_list()
f_type = df.dtypes

# Define a custom analyzer
custom_analyzer = {
    "tokenizer": "my_tokenizer",
    "filter": ["lowercase", "trim", "stop", "my_filter", "nori_filter"]
    # "char_filter": ["html_strip"]
}

# Define a custom filter for the analyzer
custom_filter = {
    "type": "pattern_replace",
    "pattern": "[^a-zA-Z0-9가-힣\\s]",
    "replacement": " "
}

# Define a custom tokenizer
custom_tokenizer = {
    "type": "nori_tokenizer",
    "decompound_mode": "mixed",
    "discard_punctuation": False
}

# nori 토큰 필터 설정
nori_filter = {
    "type": "nori_part_of_speech",
    "stoptags": ["E", "IC", "J", "MAG", "MAJ", "MM", "SP", "SSC", "SSO", "SC", "SE", "XPN", "XSA", "XSN", "XSV", "UNA", "NA", "VSV"]
}

# Create an index with custom analyzer and tokenizer settings
index_settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "my_analyzer": custom_analyzer
            },
            "filter": {
                "my_filter": custom_filter,
                "nori_filter": nori_filter
            },
            "tokenizer": {
                "my_tokenizer": custom_tokenizer
            }
        }
    },
    "mappings": {
        "properties": {}  # We'll add the properties dynamically
    }
}

# Iterate through the dataframes and dynamically add properties to the mappings
# for df in dataframes:
fields = df.columns.tolist()

for field in fields:
    field_type = "text" if pd.api.types.is_string_dtype(df[field]) or pd.api.types.is_object_dtype(df[field]) else "long"
    if field_type == "text":
        index_settings["mappings"]["properties"][field] = {"type": field_type, "analyzer": "my_analyzer"}
    else:
        index_settings["mappings"]["properties"][field] = {"type": field_type}

index_settings["mappings"]

# Create the index with the defined settings
es.indices.create(index = index_name, body = index_settings)

# Bulk insert data into Elasticsearch
actions = []
for _, row in df.iterrows():
    data = row.to_dict()
    actions.extend([{"_index": index_name, "_source": data}])

helpers.bulk(es, actions)


c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\jieun\anaconda3\envs\elastic\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.219'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


(226, [])

In [ ]:
# 인덱스 이름 설정
index_name = '과제'  # 기존 인덱스 이름을 지정하세요.

# 인덱스에 Nori 토크나이저와 분석기 설정 적용
es.indices.close(index=index_name)  # 인덱스를 닫아야 설정을 변경할 수 있습니다.
es.indices.put_settings(body=nori_settings, index=index_name)
es.indices.open(index=index_name)  # 인덱스를 다시 열어 설정을 적용합니다.

print(f"Nori 토크나이저가 '{index_name}' 인덱스에 적용되었습니다.")

c:\Users\jieun\anaconda3\envs\pykg2vec\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'settings' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  


Nori 토크나이저가 'my-index' 인덱스에 적용되었습니다.


In [ ]:
# 검색할 쿼리 설정
search_query = {
    "query": {
        "match": {
            "전철역명": "홍대입구"  # 필드 이름과 검색어를 적절하게 수정하세요.
        }
    }
}

# Elasticsearch에서 인덱스 검색 실행
index_name = 'fulltext-db'
search_results = es.search(index=index_name, body=search_query)

# 검색 결과 출력
for hit in search_results['hits']['hits']:
    source = hit['_source']
    print(source)

{'\ufeff호선': '02호선', '상/하행선': '2', '요일': '2', '전철역코드': '239', '외부코드': '239', '전철역명': '홍대입구', '첫차시간': '53000', '첫차출발역코드': '239', '첫차도착역코드': '211', '첫차출발역명': '홍대입구', '첫차도착역명': '성수', '막차시간': '235100', '막차출발역코드': '211', '막차도착역코드': '234', '막차출발역명': '성수', '막차도착역명': '신도림'}
{'\ufeff호선': '02호선', '상/하행선': '1', '요일': '3', '전철역코드': '239', '외부코드': '239', '전철역명': '홍대입구', '첫차시간': '53000', '첫차출발역코드': '239', '첫차도착역코드': '211', '첫차출발역명': '홍대입구', '첫차도착역명': '성수', '막차시간': '234900', '막차출발역코드': '211', '막차도착역코드': '202', '막차출발역명': '성수', '막차도착역명': '을지로입구'}
{'\ufeff호선': '02호선', '상/하행선': '2', '요일': '1', '전철역코드': '239', '외부코드': '239', '전철역명': '홍대입구', '첫차시간': '53000', '첫차출발역코드': '239', '첫차도착역코드': '211', '첫차출발역명': '홍대입구', '첫차도착역명': '성수', '막차시간': '245000', '막차출발역코드': '211', '막차도착역코드': '234', '막차출발역명': '성수', '막차도착역명': '신도림'}
{'\ufeff호선': '02호선', '상/하행선': '1', '요일': '2', '전철역코드': '239', '외부코드': '239', '전철역명': '홍대입구', '첫차시간': '53000', '첫차출발역코드': '239', '첫차도착역코드': '211', '첫차출발역명': '홍대입구', '첫차도착역명': '성수', '막차시간': '234900',

c:\Users\jieun\anaconda3\envs\pykg2vec\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  if sys.path[0] == "":


In [ ]:
# # 예제 데이터 생성
# example_text = "엘라스틱서치는 고성능 오픈소스 분산 검색 및 분석 엔진입니다."
# example_data = {"text": example_text}

# # 인덱스 이름 설정 (이전 예제에서 설정한 인덱스 이름을 사용하세요.)
# index_name = 'index-testing'

# # 예제 데이터를 Elasticsearch에 색인화
# es.index(index=index_name, body=example_data)

# Elasticsearch에서 Nori 토크나이저 및 분석기를 사용하여 분석 결과 확인
analysis_request = {
    "text": '남부터미널'
}

analysis_result = es.indices.analyze(index=index_name, body=analysis_request)

# 분석 결과 출력
tokens = [token['token'] for token in analysis_result['tokens']]
# print("원본 텍스트:", example_text)
print("Nori 토큰화 결과:", " ".join(tokens))


Nori 토큰화 결과: 남부터미널


c:\Users\jieun\anaconda3\envs\pykg2vec\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  


In [ ]:
print(analysis_result['tokens'])

[{'token': '남부터미널', 'start_offset': 0, 'end_offset': 5, 'type': '<HANGUL>', 'position': 0}]


---

In [ ]:
nori+tokenizer를 이용한 한글 형태소 분석하기
/*
 * [인덱스 생성 및 설정]
*/
PUT nori_sample
{
  "settings": {
    "index": {
      // elasticsearch에서 기본 제공하는 설정, 색인에 대한 분석 구성을 정의한다.
      "analysis": {
        // 색인에 대한 토크나이저를 구성한다.
        "tokenizer": {
          "my_nori_tokenizer": {
            "type": "nori_tokenizer",
            "decompound_mode": "mixed", // 'mixed' 복합 명사를 분해하는 기능을 수행
            "discard_punctuation": "false" // 문장 부호 제거하지 않도록 설정한다. (기본적으로 제거한다.)
          }
        },
        // analyzer는 분할된 토큰에 필터링과 정규화 처리 통헤 섹인에 추가할 단어의 집합을 만드는 데 사용된다.
        "analyzer": {
          // analyzer를 여러 개 만들어서 사용 가능하다.
          "my_nori_analyzer": {
            "type": "custom",
            "tokenizer": "my_nori_tokenizer", 
            // lowercase: 알파벳 소문자로 통일 시킨다.
            // stop: 문서에서 자주 사용되는 불용어를 제거한다.(검색 품질 향상: "the", "a", "an", "and" 와 같은 단어 검색에서 제외)
            "filter": ["lowercase", "stop"],
            // html_strip: HTML 태그 제거
            "char_filter": ["html_strip"]
          }
        }
      }
    }
  },
  "mappings" : {
    "properties" : {
      "title": {
        "type" : "text",
        "analyzer": "my_nori_analyzer"
      }
    }
  }
}

In [ ]:
GET nori_sample/_analyze
{
  "tokenizer": "my_nori_tokenizer",
  "text": [
    "동해물과 백두산이"
  ]
}

# 엘라스틱서치에는 여러개의 필드타입이 존재하는데요 text 타입일때 analyzer를 통한 fulltext 검색이 가능합니다
# 흔히 text와 keyword 타입의 차이가 뭐지라는 질문이 있는데요
# text는 풀텍스트 서치가 필요한 텍스트,
# keyword는 구조화된 텍스트, 예를들면 ID나 이메일, 우편번호같이 풀텍스트 서치가 필요없는 필드를 지정할때 쓰시면 됩니다

In [ ]:
# 생성된 인덱스의 매핑 가져오기
GET /doc1/_mapping

# 
PUT /_template/nori_template
{
  "index_patterns": ["doc1", "doc2"],
  "settings": {
    "analysis": {
      "analyzer": {
        "nori_sample_analyzer": {
          "type": "custom",
          "tokenizer": "nori_tokenizer"
        }
      },
      "tokenizer": {
        "nori_tokenizer": {
          "type": "nori_tokenizer",
          "decompound_mode": "mixed"
        }
      }
    }
  }
}

# 

---

In [6]:
# search_app/setting_bulk.py   # 여기에 구현
# 인덱스 생성
es.indices.create( 
    index='dictionary',  # 인덱스 이름: dictionary
    body={
        "settings": {
            "index": { 
                "analysis": {
                    "analyzer": {  # 애널라이저 put
                        "my_analyzer": {  # 애널라이저 이름
                            "type": "custom",
                            "tokenizer": "nori_tokenizer"  # 노리 적용
                        }
                    }
                }
            }
        }
    }
)

c:\Users\jieun\anaconda3\envs\pykg2vec\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  # Remove the CWD from sys.path while we load stuff.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'dictionary'})

In [ ]:
# search_app/setting_bulk.py   # 여기에 구현
# 매핑 설정 (인덱스에 들어가는 데이터의 타입을 정의. 자동으로 만들어주는 것보다 지정하는 것을 권장.)
es.indices.create(
    index='dictionary',  # dictionary 인덱스에
    body={
        "settings": {
            "index": {
                "analysis": {
                    "analyzer": {
                        "my_analyzer": {
                            "type": "custom",
                            "tokenizer": "nori_tokenizer"
                        }
                    }
                }
            }
        },
        "mappings": {  # 매핑을 설정
            "dictionary_datas": {
                "properties": {
                    "id": {
                        "type": "long"
                    },
                    "title": {
                        "type": "text",  # 타입 텍스트로 지정
                        "analyzer": "my_analyzer"
                    },
                    "content": {
                        "type": "text",  # 타입 텍스트로 지정
                        "analyzer": "my_analyzer"
                    }
                }
            }
        }
    }
)

In [11]:
# search_app/setting_bulk.py
# bulk json version

# import json

# with open("dictionary_data.json", encoding='utf-8') as json_file:
#     json_data = json.loads(json_file.read())

# body = ""
# for i in json_data:
#     body = body + json.dumps({"index": {"_index": "dictionary", "_type": "dictionary_datas"}}) + '\n'
#     body = body + json.dumps(i, ensure_ascii=False) + '\n'

# es.bulk(body)

FileNotFoundError: [Errno 2] No such file or directory: 'dictionary_data.json'

In [12]:
# search_app/views.py
search_word = request.query_params.get('search')

if not search_word:
    return Response(status=status.HTTP_400_BAD_REQUEST, data={'message': 'search word param is missing'})

docs = es.search(index='dictionary',
                    doc_type='dictionary_datas',
                    body={
                        "query": {
                            "multi_match": {
                                "query": search_word,
                                "fields": ["title", "content"]
                            }
                        }
                    })

data_list = docs['hits']
data_list

ModuleNotFoundError: No module named 'rest_framework'

In [13]:
# server_project/urls.py
# uri 설정. url의 설정한 부분까지 잘라내고 남은 문자열 부분의 후속 처리를 위해 search_app의 urls.py와 연결해줍니다.
from django.contrib import admin  
from django.urls import path  
  
from django.conf.urls import include  
  
urlpatterns = [  
    path('admin/', admin.site.urls),  
    path('', include('search_app.urls')),  
]

ERROR: Could not find a version that satisfies the requirement rest_framework (from versions: none)
ERROR: No matching distribution found for rest_framework


In [ ]:
# search_app/urls.py
# 이제 Django는 http://127.0.0.1:8000/로 들어오는 모든 접속 요청을 search_app.urls로 전송해 추가 명령을 찾을 것입니다.
# search_app 디렉터리에 urls.py 파일을 생성하고 url 패턴을 추가해줍니다.
# 참고: https://blog.nerdfactory.ai/2019/04/29/django-elasticsearch-restframework.html
from django.urls import path  
from search_app import views  
  
urlpatterns = [  
    path('', views.SearchView.as_view()),  
]

In [64]:
import pandas as pd
from pandas.api.types import is_string_dtype

# 사용자로부터 필드 정보 입력받기
fields = []
print(fields)


dd = pd.read_csv("C:/Users/jieun/Desktop/project/elastic/data/3_서울시 종로구 모범음식점 지정 현황.csv")
dd

[]


,시군구코드,지정년도,지정번호,신청일자,지정일자,업소명,소재지도로명,소재지지번,허가(신고)번호,업태명,주된음식,영업장면적(㎡),행정동명,급수시설구분,소재지전화번호
0,3000000,2021,13,20211025,20211230,늘마중,"서울특별시 종로구 인사동10길 11-5, 1층 (관훈동)",서울특별시 종로구 관훈동 30번지 16호,3000000-101-2021-00138,한식,"막걸리, 전",59.50,종로1.2.3.4가동,NaN,02 730 2985
1,3000000,2021,90,20211126,20211216,끄티집,"서울특별시 종로구 삼일대로19길 20, 1~4층 (관철동)",서울특별시 종로구 관철동 14번지 5호,3000000-101-2019-00441,한식,"전골, 샤브샤브",445.45,종로1.2.3.4가동,NaN,02725 7772
2,3000000,2021,89,20211025,20211216,말뚜기 감자탕,"서울특별시 종로구 종로51길 23-9, 1층 (창신동)",서울특별시 종로구 창신동 581번지 30호 1층,3000000-101-2016-00071,식육(숯불구이),감자탕,48.92,창신제1동,NaN,NaN
3,3000000,2021,88,20210311,20210325,스윗샐러드,"서울특별시 종로구 종로1길 50, 더케이트윈타워 지하1층 (중학동)",서울특별시 종로구 중학동 19번지 더케이트윈타워,3000000-101-2018-00205,경양식,샐러드,64.81,종로1.2.3.4가동,NaN,NaN
4,3000000,2019,15,20191030,20191231,대가곱창,"서울특별시 종로구 명륜길 53, (명륜3가,(지상1층))",서울특별시 종로구 명륜3가 1번지 1143호 (지상1층),3000000-101-2009-00015,한식,"곱창, 막창",29.37,혜화동,NaN,02 747 3827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,3000000,2007,9,20071112,20071120,진고집 동해 동태탕,"서울특별시 종로구 지봉로 26-1, 1층 (숭인동)",서울특별시 종로구 숭인동 313번지 8호,3000000-101-2003-00437,한식,곰탕,158.40,숭인제2동,NaN,02 7780554
86,3000000,2007,84,20071109,20071120,향가,"서울특별시 종로구 계동길 19-6, (재동)",서울특별시 종로구 재동 84번지 1호,3000000-101-2000-11447,한식,시골밥상,200.65,가회동,상수도전용,02 7473368
87,3000000,2007,5,20071112,20071120,궁나라냉면.묵밥,"서울특별시 종로구 지봉로12길 6, 1층 (숭인동)",서울특별시 종로구 숭인동 56번지 38호,3000000-101-1992-01124,한식,"냉면,묵밥",115.50,숭인제1동,상수도전용,02744 4701
88,3000000,2007,19,20071112,20071120,더레스토랑,"서울특별시 종로구 삼청로 54, 지하1,지상1,2층 (소격동)",서울특별시 종로구 소격동 48번지,3000000-101-1999-10390,경양식,스테이크,408.19,삼청동,상수도전용,02 7358441


In [65]:
from elasticsearch.helpers import bulk
from elasticsearch.client import IndicesClient

# CSV 파일 불러오기
# data_path = "data/"
# csv_files = os.listdir(data_path)
# dataframes = [pd.read_csv(data_path + file) for file in csv_files]
# print(csv_files)

dd = pd.read_csv("C:/Users/jieun/Desktop/project/elastic/data/3_서울시 종로구 모범음식점 지정 현황.csv")
dd = dd.fillna(0)
index_name = "서울시_종로구_모범음식점_지정_현황"

# Elasticsearch 인덱스 생성
indices_client = IndicesClient(es)

field_name = dd.columns.to_list()
f_type = dd.dtypes

fields = []
for i, field in enumerate(field_name):
    field_name = field
    field_type = f_type[i]
    # print(field_type)
    if is_string_dtype(field_type):
        fields.append({
            "name": field_name,
            "type": "text",            # 문자형은 text 타입으로
            "analyzer": "my_analyzer"
        })
    else:
        fields.append({
            "name": field_name,
            "type": "long"             # 숫자형은 long 타입으로
        })

# 매핑 설정
mappings = {
    "mappings": {
        "dictionary_datas": {
            "properties": {}
        }
    },
    "settings": {
        "index": {
            "analysis": {
                "analyzer": {
                    "my_analyzer": {     ## analyzer 이름 지정
                        "type": "custom",
                        "tokenizer": "nori_tokenizer",
                        "filter": ["lowercase", "trim", "stop"],
                        "char_filter": ["html_strip"],
                        "filter": ["my_filter"]
                    }
                },
                "filter":{
                    "my_filter":{
                        "type":"pattern_replace",
                        "pattern": "[^a-zA-Z0-9가-힣\\s]",
                        "replacement": ""
                    }  
                },
                "tokenizer": {           ## tokenizer 이름 지정
                    "my_tokenizer": {
                        "type": "nori_tokenizer",
                        "decompound_mode": "mixed"
                    }
                }
            }
        }
    }
}

# 사용자가 입력한 필드 정보를 기반으로 매핑 설정 추가
for field_info in fields:
    field_name = field_info["name"]
    field_type = field_info["type"]

    if field_type == 'text':
        analyzer_name = field_info["analyzer"]
        mappings["mappings"]["dictionary_datas"]["properties"][field_name] = {
        "type": field_type,
        "analyzer": "my_analyzer"
    }
    else:
        mappings["mappings"]["dictionary_datas"]["properties"][field_name] = {
        "type": field_type
    }

# 인덱스 생성 및 매핑 설정 적용
indices_client.create(index=index_name, body={"settings": mappings})

# CSV 데이터를 Elasticsearch에 업로드
data = dataframe.to_dict(orient='records')
actions = [{"_index": index_name, "_source": item} for item in data]
print(actions)
bulk(es, actions)

C:\Users\jieun\AppData\Local\Temp\ipykernel_43116\3336312759.py:2: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import IndicesClient


NameError: name 'es' is not defined

In [57]:
# 매핑 설정
mappings = {
    "mappings": {
        "dictionary_datas": {
            "properties": {}
        }
    },
    "settings": {
            "index": {
                "analysis": {
                    "analyzer": {
                        "my_analyzer": {
                            "type": "custom",
                            "tokenizer": "nori_tokenizer"
                        }
                    }
                }
            }
        }
}

# 사용자가 입력한 필드 정보를 기반으로 매핑 설정 추가
for field_info in fields:
    field_name = field_info["name"]
    field_type = field_info["type"]

    if is_string_dtype(field_type):
        analyzer_name = field_info["analyzer"]
        mappings["mappings"]["dictionary_datas"]["properties"][field_name] = {
        "type": field_type,
        "analyzer": analyzer_name
    }
    else:
        mappings["mappings"]["dictionary_datas"]["properties"][field_name] = {
        "type": field_type
    }

mappings

{'mappings': {'dictionary_datas': {'properties': {'시군구코드': {'type': 'long'},
    '지정년도': {'type': 'long'},
    '지정번호': {'type': 'long'},
    '신청일자': {'type': 'long'},
    '지정일자': {'type': 'long'},
    '업소명': {'type': 'text'},
    '소재지도로명': {'type': 'text'},
    '소재지지번': {'type': 'text'},
    '허가(신고)번호': {'type': 'text'},
    '업태명': {'type': 'text'},
    '주된음식': {'type': 'text'},
    '영업장면적(㎡)': {'type': 'long'},
    '행정동명': {'type': 'text'},
    '급수시설구분': {'type': 'text'},
    '소재지전화번호': {'type': 'text'}}}},
 'settings': {'index': {'analysis': {'analyzer': {'my_analyzer': {'type': 'custom',
      'tokenizer': 'nori_tokenizer'}}}}}}

---

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q -U bitsandbytes

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# model_id = "EleutherAI/polyglot-ko-12.8b"
# bnb_config = BitsAndBytesConfig(
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)